<a href="https://colab.research.google.com/github/GavinButts/Fall-Data-Challenge-2022/blob/main/Code/Kevin/Ordinal%20-%20SEGRADES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
!pip install --upgrade --no-deps statsmodels
import scipy.stats as stats


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 6.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [147]:
sheet_id = "11rno0SjsOyV1JlevH3q40UMm1s5e6m7g"
sheet_name = "curated_2019-required"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url)
print(df)

            BASMID  ALLGRADEX  EDCPUB  SCCHOICE  SPUBCHOIX  SCONSIDR  \
0      20191000012         12       1         2          3         2   
1      20191000029          4       1         2          3         2   
2      20191000059         11       2         2          2         1   
3      20191000070          6       1         1          1         1   
4      20191000078          9       1         2          3         2   
...            ...        ...     ...       ...        ...       ...   
15495  20191225472          7       1         2          2         2   
15496  20191225475          2       1         2          2         2   
15497  20191225477         12       1         1          3         2   
15498  20191225479         13       1         2          2         1   
15499  20191225500         14       1         1          2         2   

       SCHLHRSWK  EINTNET  MOSTIMPT  INTNUM  ...  SEFUTUREX  DSBLTY  \
0              4        4        -1      -1  ...          6     

In [149]:
#create FOYESCNT variable that determines the amount of times each person said 'Yes' on the FO#### questions (ITEMS 72-74)
#Legend:  Number of 'Yes'

FOYESCNT = []      #variable that will be added to csv
yesCount = 0
for i in range(15500):
  if df['FOSTORY2X'][i] == 1:
    yesCount += 1
  if df['FOCRAFTS'][i] == 1:
    yesCount += 1
  if df['FOGAMES'][i] == 1:
    yesCount += 1
  if df['FOBUILDX'][i] == 1:
    yesCount += 1
  if df['FOSPORT'][i] == 1:
    yesCount += 1
  if df['FORESPON'][i] == 1:
    yesCount += 1
  if df['FOHISTX'][i] == 1:
    yesCount += 1
  if df['FOLIBRAYX'][i] == 1:
    yesCount += 1
  if df['FOBOOKSTX'][i] == 1:
    yesCount += 1
  if df['FODINNERX'][i] != 0:
    yesCount += 1
  if df['FHCHECKX'][i] > 1:
    yesCount += 1
  if df['FHHELP'][i] > 1 and df['FHHELP'][i] < 5:
    yesCount += 1

  FOYESCNT.append(yesCount)
  yesCount = 0

df["FOYESCNT"] = FOYESCNT
df.to_csv("sample.csv", index=False)  #adds variable to csv

#create FSYESCNT variable that determines the amount of times each person said 'Yes' on the FS#### questions (ITEMS 60)
#Legend:  Number of 'Yes'

FSYESCNT = []      #variable that will be added to csv
yesCount = 0
FSYESCNT_LMH = []

for i in range(15500):
  if df['FSSPORTX'][i] == 1:
    yesCount += 1
  if df['FSVOL'][i] == 1: #bad
    yesCount += 1
  if df['FSMTNG'][i] == 1: #bad
    yesCount += 1
  if df['FSPTMTNG'][i] == 1:
    yesCount += 1
  if df['FSATCNFN'][i] == 1:
    yesCount += 1
  if df['FSFUNDRS'][i] == 1:
    yesCount += 1
  if df['FSCOMMTE'][i] == 1: #bad
    yesCount += 1
  if df['FSCOUNSLR'][i] == 1:
    yesCount += 1
  if df['FSFREQ'][i] > 0: #bad
    yesCount += 1
  if df['FSNOTESX'][i] == 1:
    yesCount += 1

    
  FSYESCNT.append(yesCount)
  if yesCount <= 3:
    FSYESCNT_LMH.append('1) LOW INVOLVMENT')
  elif yesCount > 3 and yesCount<=5:
    FSYESCNT_LMH.append('2) MEDIUM INVOLVEMENT')
  elif yesCount > 5:
    FSYESCNT_LMH.append('3) HIGH INVOLVEMENT')
  yesCount = 0

df["FSYESCNT"] = FSYESCNT
df["FSYESCNT_LMH"] = FSYESCNT_LMH

df.to_csv("sample.csv", index=False)  #adds variable to csv


In [150]:
# delete all rows for students who answered 5 or -1
'''
df_clean = []
for i in range(len(df)):
  if !(df.SEGRADES[i] == -1 or df.SEGRADES[i] == 5):
    df_clean.append(df[]) = df.drop(i)
'''


i = 0
L = len(df.SEGRADES)
while i < L:
  if ((df.SEGRADES[i] == -1) or (df.SEGRADES[i] == 5)):
    df = df.drop(i)
  i+=1


In [151]:
print(df['SEGRADES'])


0        1
2        3
3        1
4        1
6        1
        ..
15494    4
15495    1
15497    1
15498    1
15499    2
Name: SEGRADES, Length: 13533, dtype: int64


In [152]:
#Data Pre-Processing
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=[1,2,3,4], ordered=True)
df["SEGRADES"] = df["SEGRADES"].astype(cat_type)
df["SEGRADES"].dtype

CategoricalDtype(categories=[1, 2, 3, 4], ordered=True)

In [153]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [175]:
#['HHPARN19X','FHPLACE','SEABSNT','FSYESCNT','SEGRADES','FOYESCNT','RACEETH','TTLHHINC']
x = ['ALLGRADEX','EDCPUB','SEGRADEQ','FOYESCNT','FSSPORTX','FSPTMTNG','HHPARN19X','TTLHHINC',
     'FSNOTESX','FSCOUNSLR','FSFUNDRS','FSATCNFN','SEABSNT','RACEETH','OWNRNTHB','SEFUTUREX']
mod_prob = OrderedModel(df['SEGRADES'],
                        df[x],
                        distr='logit')
 
res_log = mod_prob.fit(method='bfgs')
res_log.summary()


Optimization terminated successfully.
         Current function value: 0.628090
         Iterations: 61
         Function evaluations: 63
         Gradient evaluations: 63


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:               SEGRADES   Log-Likelihood:                -8499.9
Model:                   OrderedModel   AIC:                         1.704e+04
Method:            Maximum Likelihood   BIC:                         1.718e+04
Date:                Fri, 28 Oct 2022                                         
Time:                        22:51:51                                         
No. Observations:               13533                                         
Df Residuals:                   13514                                         
Df Model:                          19                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ALLGRADEX      0.0812      0.007     11.406      0.000       0.067       0.095
EDCPUB        -0.2442      0.072     -3.394      0.001      -0.385      -0.103
SEGRADEQ       2.1876      0.034     64.943      0.000       2.122       2.254
FOYESCNT       0.0366      0.009      3.937      0.000       0.018       0.055
FSSPORTX       0.2689      0.051      5.230      0.000       0.168       0.370
FSPTMTNG      -0.1292      0.045     -2.903      0.004      -0.216      -0.042
HHPARN19X      0.1842      0.026      7.080      0.000       0.133       0.235
TTLHHINC      -0.0576      0.008     -7.038      0.000      -0.074      -0.042
FSNOTESX      -0.1915      0.044     -4.350      0.000      -0.278      -0.105
FSCOUNSLR     -0.2860      0.044     -6.516      0.000      -0.372      -0.200
FSFUNDRS       0.2439      0.045      5.477      0.000       0.157       0.331
FSATCNFN      -0.1212      0.049     -2.450      0.014      -0.218      -0.024
SEABSNT        0.2516      0.036      7.064      0.000       0.182       0.321
RACEETH        0.0648      0.017      3.861      0.000       0.032       0.098
OWNRNTHB       0.0971      0.046      2.095      0.036       0.006       0.188
SEFUTUREX     -0.2001      0.019    -10.774      0.000      -0.236      -0.164
1/2            4.5481      0.247     18.394      0.000       4.063       5.033
2/3            1.1850      0.015     81.453      0.000       1.156       1.214
3/4            1.0634      0.027     39.203      0.000       1.010       1.117
==============================================================================
"""

In [176]:
predicted = res_log.model.predict(res_log.params, exog=df[x])
predicted


/usr/local/lib/python3.7/dist-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


array([[9.34758253e-01, 6.25980047e-02, 2.49735448e-03, 1.46387428e-04],
       [1.13039889e-02, 2.20081063e-01, 6.13585644e-01, 1.55029304e-01],
       [9.08670457e-01, 8.75268376e-02, 3.59191345e-03, 2.10792140e-04],
       ...,
       [3.33308743e-01, 5.96088463e-01, 6.64244827e-02, 4.17831064e-03],
       [8.69003925e-01, 1.25303618e-01, 5.37634533e-03, 3.16111524e-04],
       [1.55897688e-01, 6.73540465e-01, 1.59331540e-01, 1.12303076e-02]])

In [177]:
pred_choice = predicted.argmax(1)
print('Fraction of correct choice predictions')
print((np.asarray(df['SEGRADES'].values.codes) == pred_choice).mean())


Fraction of correct choice predictions
0.721347816448681
